In [1]:
import numpy as np
import json
import gzip
from scipy.sparse import coo_matrix
import pandas as pd
import pickle
from collections import defaultdict

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import SimpleConv
from torch_geometric.data import Data

c:\Users\rebal\capstone-project\capstone\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class DEHNNLayer(nn.Module):
    def __init__(self, node_in_features, edge_in_features, vn_features):
        super(DEHNNLayer, self).__init__()
        self.node_mlp1 = nn.Sequential(nn.Linear(edge_in_features, edge_in_features),
                                       nn.ReLU(),
                                       nn.Linear(edge_in_features, edge_in_features))
        
        self.edge_mlp2 = nn.Sequential(nn.Linear(node_in_features, node_in_features),
                                       nn.ReLU(),
                                       nn.Linear(node_in_features, node_in_features))
        
        self.edge_mlp3 = nn.Sequential(nn.Linear(2 * node_in_features, 2 * node_in_features),
                                       nn.ReLU(),
                                       nn.Linear(2 * node_in_features, 2 * node_in_features))

        self.node_to_virtual_mlp = nn.Sequential(nn.Linear(node_in_features, vn_features),
                                       nn.ReLU(),
                                       nn.Linear(vn_features, vn_features))
        
        self.virtual_to_higher_virtual_mlp = nn.Sequential(nn.Linear(vn_features, vn_features),
                                       nn.ReLU(),
                                       nn.Linear(vn_features, vn_features))
        
        self.higher_virtual_to_virtual_mlp = nn.Sequential(nn.Linear(vn_features, vn_features),
                                       nn.ReLU(),
                                       nn.Linear(vn_features, vn_features))
        
        self.virtual_to_node_mlp = nn.Sequential(nn.Linear(vn_features, vn_features),
                                       nn.ReLU(),
                                       nn.Linear(vn_features, edge_in_features))

        # Learnable defaults for missing driver or sink
        self.default_driver = nn.Parameter(torch.zeros(node_in_features))
        self.default_sink_agg = nn.Parameter(torch.zeros(node_in_features))
        self.default_edge_agg = nn.Parameter(torch.zeros(edge_in_features))

    def forward(self, node_features, edge_features, vn_features, super_vn_features, hypergraph):
        # Node update
        updated_node_features = {}
        for node in range(hypergraph.num_nodes):
            incident_edges = hypergraph.get_incident_edges(node)
            if incident_edges:
                agg_features = torch.sum(torch.stack([self.node_mlp1(edge_features[edge]) for edge in incident_edges]), dim=0)
            else:
                agg_features = self.default_edge_agg  # Fallback for isolated nodes
            updated_node_features[node] = agg_features

        # Edge update
        updated_edge_features = {}
        for edge in range(hypergraph.num_edges):
            driver, sinks = hypergraph.get_driver_and_sinks(edge)

            # Handle missing driver
            driver_feature = node_features[driver] if driver is not None else self.default_driver

            # Handle missing sinks
            if sinks:
                sink_agg = torch.sum(torch.stack([self.edge_mlp2(node_features[sink]) for sink in sinks]), dim=0)
            else:
                sink_agg = self.default_sink_agg

            # Concatenate and update
            concatenated = torch.cat([driver_feature, sink_agg])
            updated_edge_features[edge] = self.edge_mlp3(concatenated)
        
        updated_vn_features = {}
        for virtual_node in range(hypergraph.num_vn):
            assigned_nodes = hypergraph.get_nodes_from_vn(virtual_node)
            agg_features = torch.sum(torch.stack([self.node_to_virtual_mlp(node_features[node]) for node in assigned_nodes]), dim=0)
            agg_features += self.higher_virtual_to_virtual_mlp(super_vn_features)
            updated_vn_features[virtual_node] = agg_features

        updated_super_vn_features = torch.sum(
            torch.stack([self.virtual_to_higher_virtual_mlp(vn_features[vn]) for vn in range(hypergraph.num_vn)]), dim=0
        )

        for node in range(hypergraph.num_nodes):
            virtual_node = hypergraph.get_vn_from_node(node)
            propagated_feature = self.virtual_to_node_mlp(vn_features[virtual_node])
            updated_node_features[node] += propagated_feature  # Add propagated feature to node

        return updated_node_features, updated_edge_features, updated_vn_features, updated_super_vn_features


class DEHNN(nn.Module):
    def __init__(self, num_layers, node_in_features, edge_in_features):
        super(DEHNN, self).__init__()
        self.num_layers = num_layers
        self.layers = nn.ModuleList()
        
        # Create multiple layers for DEHNN
        vn_in_features = node_in_features
        for i in range(num_layers):
            self.layers.append(DEHNNLayer(node_in_features, edge_in_features, vn_in_features))
            node_in_features, edge_in_features = edge_in_features, node_in_features
            edge_in_features *= 2

        edge_in_features  = int(edge_in_features / 2)
        self.output_layer = nn.Linear(node_in_features, 1)

    def forward(self, node_features, edge_features, vn_features, super_vn_features, hypergraph):
        # Pass through each layer
        for layer in self.layers:
            node_features, edge_features, vn_features, super_vn_features = layer(node_features, edge_features, vn_features, super_vn_features, hypergraph)
        
        # Output prediction for nodes
        final_node_features = torch.stack([node_features[node] for node in range(hypergraph.num_nodes)], dim=0)
        output = self.output_layer(final_node_features)
        return output


# Example hypergraph representation class (simplified)
class Hypergraph:
    def __init__(self, num_nodes, num_edges, num_vn, driver_sink_map, node_to_virtual, virtual_to_node, incidence_matrix):
        self.num_nodes = num_nodes
        self.num_edges = num_edges
        self.num_vn = num_vn
        self.driver_sink_map = driver_sink_map
        self.node_to_virtual = node_to_virtual
        self.virtual_to_node = virtual_to_node
        self.incidence_matrix = incidence_matrix

    def get_incident_edges(self, node):
        return [edge for edge in range(self.num_edges) if node in self.driver_sink_map[edge][1] or node == self.driver_sink_map[edge][0]]

    def get_driver_and_sinks(self, edge):
        return self.driver_sink_map[edge]
    
    def get_vn_from_node(self, node):
        return self.node_to_virtual[node]
    
    def get_nodes_from_vn(self, vn):
        return self.virtual_to_node[vn]

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

clean_data_dir = '../../data/chips/clean_data/'

with open(clean_data_dir + '1.driver_sink_map.pkl', 'rb') as f:
    driver_sink_map = pickle.load(f)

with open(clean_data_dir + '1.node_features.pkl', 'rb') as f:
    node_features = pickle.load(f)

with open(clean_data_dir + '1.net_features.pkl', 'rb') as f:
    edge_features = pickle.load(f)

with open(clean_data_dir + '1.congestion.pkl', 'rb') as f:
    congestion = pickle.load(f)

partition = np.load(clean_data_dir + '1.partition.npy')
conn = np.load('../../data/chips/NCSU-DigIC-GraphData-2023-07-25/xbar/1/xbar_connectivity.npz')

incidence_matrix = torch.sparse_coo_tensor(torch.tensor(np.array([conn['row'], conn['col']])), torch.ones(conn['data'].shape), dtype=torch.float).to(device)

node_features = torch.tensor(np.array(list(node_features.values())), dtype=torch.float).to(device)
edge_features = torch.tensor(np.array(list(edge_features.values())), dtype=torch.float).to(device)

num_nodes, num_node_features = node_features.shape
num_edges, num_edge_features = edge_features.shape

node_to_virtual = {i: p for i, p in enumerate(partition)}
virtual_to_node = defaultdict(list)
for i, p in enumerate(partition):
    virtual_to_node[p].append(i)

num_vn = len(virtual_to_node)
vn_features = torch.zeros((num_vn, num_node_features), dtype=torch.float).to(device)
super_vn_features = torch.zeros(num_node_features, dtype=torch.float).to(device)

nodes = list(range(len(node_features)))
edges = list(range(len(edge_features)))
hypergraph = Hypergraph(num_nodes, num_edges, num_vn, driver_sink_map, node_to_virtual, virtual_to_node, incidence_matrix)

In [4]:
incidence_matrix.to_dense()

tensor([[0., 0., 0.,  ..., 0., 0., 1.],
        [0., 0., 0.,  ..., 0., 0., 1.],
        [0., 0., 0.,  ..., 0., 0., 1.],
        ...,
        [0., 1., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 1.],
        [1., 0., 0.,  ..., 0., 0., 1.]], device='cuda:0')

In [114]:
model.layers[0]._modules['node_mlp1'][0].weight

Parameter containing:
tensor([[-0.0796]], device='cuda:0', requires_grad=True)

In [110]:
model.layers[0]._modules['node_mlp1'](torch.tensor([[42]], dtype=torch.float).to(device))

tensor([[0.8652]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [ ]:
model.layers[0]._modules['node_mlp1'](torch.tensor([[0.8652]], dtype=torch.float).to(device))

In [100]:
edge_features

tensor([[ 1.],
        [42.],
        [ 1.],
        ...,
        [33.],
        [33.],
        [14.]], device='cuda:0')

In [27]:
# Initialize DE-HNN model
model = DEHNN(num_layers=4, node_in_features=num_node_features, edge_in_features=num_edge_features).to(device)
epochs = 1

# Optimizer and Loss Function
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.L1Loss()

# Training Loop (example)
model.train()
for epoch in range(epochs):
    optimizer.zero_grad()
    
    # Forward pass
    output = model(node_features, edge_features, vn_features, super_vn_features, hypergraph)
    output = output[:,0]
    # print(output)
    
    # Dummy target for illustration (binary labels for each node: 0 for not congested, 1 for congested)
    target = torch.tensor(list(congestion.values())).to(device)
    
    # print(target)
    # Compute loss
    loss = criterion(output, target)
    
    # Backward pass
    loss.backward()
    optimizer.step()
    
    # Print loss
    print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

Epoch [1/1], Loss: 78.7692


In [28]:
model.layers

ModuleList(
  (0): DEHNNLayer(
    (node_mlp1): Sequential(
      (0): Linear(in_features=1, out_features=1, bias=True)
      (1): ReLU()
      (2): Linear(in_features=1, out_features=1, bias=True)
    )
    (edge_mlp2): Sequential(
      (0): Linear(in_features=14, out_features=14, bias=True)
      (1): ReLU()
      (2): Linear(in_features=14, out_features=14, bias=True)
    )
    (edge_mlp3): Sequential(
      (0): Linear(in_features=28, out_features=28, bias=True)
      (1): ReLU()
      (2): Linear(in_features=28, out_features=28, bias=True)
    )
    (node_to_virtual_mlp): Sequential(
      (0): Linear(in_features=14, out_features=14, bias=True)
      (1): ReLU()
      (2): Linear(in_features=14, out_features=14, bias=True)
    )
    (virtual_to_higher_virtual_mlp): Sequential(
      (0): Linear(in_features=14, out_features=14, bias=True)
      (1): ReLU()
      (2): Linear(in_features=14, out_features=14, bias=True)
    )
    (higher_virtual_to_virtual_mlp): Sequential(
      (0)

In [25]:
out = output.detach().cpu().numpy()
real = target.detach().cpu().numpy()

lst = [0, 0, 0, 0]
for i, j in zip(out, real):
    if i >= 0.9 and j >= 0.9:
        lst[0] += 1
    elif i < 0.9 and j >= 0.9:
        lst[1] += 1
    elif i >= 0.9 and j < 0.9:
        lst[2] += 1
    else:
        lst[3] += 1
total = sum(lst)
print(lst)
print(f'Accuracy: {(lst[0] + lst[3]) / total}')
print(f'Precision: {lst[0] / (lst[0] + lst[1])}')
print(f'Accuracy: {lst[0] / (lst[0] + lst[2])}')

[0, 396, 1, 3555]
Accuracy: 0.8995445344129555
Precision: 0.0
Accuracy: 0.0


In [20]:
out

array([0.20099005, 0.20232299, 0.17674008, ..., 0.07204804, 0.02181804,
       0.00367373], dtype=float32)

In [21]:
real

array([0.83333333, 0.72222222, 0.83333333, ..., 0.57142857, 0.69444444,
       0.6969697 ])

In [ ]:
# Fast train

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

clean_data_dir = '../../data/chips/clean_data/'

with open(clean_data_dir + '1.driver_sink_map.pkl', 'rb') as f:
    driver_sink_map = pickle.load(f)

with open(clean_data_dir + '1.node_features.pkl', 'rb') as f:
    node_features = pickle.load(f)

with open(clean_data_dir + '1.net_features.pkl', 'rb') as f:
    edge_features = pickle.load(f)

with open(clean_data_dir + '1.congestion.pkl', 'rb') as f:
    congestion = pickle.load(f)

partition = np.load(clean_data_dir + '1.partition.npy')
conn = np.load(clean_data_dir + '1.connectivity.npz')

incidence_matrix = torch.sparse_coo_tensor(torch.tensor(np.array([conn['row'], conn['col']])), torch.ones(conn['dirs'].shape), dtype=torch.float).to(device)

idx = conn['dirs'] == 1
driver_matrix = torch.sparse_coo_tensor(torch.tensor(np.array([conn['col'][idx], conn['row'][idx]])), torch.ones(conn['dirs'][idx].shape), dtype=torch.float).to(device)

idx = conn['dirs'] == 0
sink_matrix = torch.sparse_coo_tensor(torch.tensor(np.array([conn['col'][idx], conn['row'][idx]])), torch.ones(conn['dirs'][idx].shape), dtype=torch.float).to(device)

node_features = torch.tensor(np.array(list(node_features.values())), dtype=torch.float).to(device)
edge_features = torch.tensor(np.array(list(edge_features.values())), dtype=torch.float).to(device)

num_nodes, num_node_features = node_features.shape
num_edges, num_edge_features = edge_features.shape

node_to_virtual = {i: p for i, p in enumerate(partition)}
virtual_to_node = defaultdict(list)
for i, p in enumerate(partition):
    virtual_to_node[p].append(i)

num_vn = len(virtual_to_node)
vn_features = torch.zeros((num_vn, num_node_features), dtype=torch.float).to(device)
super_vn_features = torch.zeros(num_node_features, dtype=torch.float).to(device)

vn_rows = []
vn_cols = []

for k, v in virtual_to_node.items():
    for n in v:
        vn_rows.append(k)
        vn_cols.append(n)

vn_matrix = torch.sparse_coo_tensor(torch.tensor(np.array([vn_rows, vn_cols])), torch.ones(len(vn_rows)), dtype=torch.float).to(device)

nodes = list(range(len(node_features)))
edges = list(range(len(edge_features)))
hypergraph = Hypergraph(num_nodes, num_edges, num_vn, driver_sink_map, node_to_virtual, virtual_to_node, incidence_matrix, driver_matrix, sink_matrix, vn_matrix)